In [1]:
#Dependencies
import requests
import json
import pandas as pd
import numpy as np
from apikey import api_key

In [2]:
#Create base urls
baseurl = "https://api.census.gov/data/"

#Create suffix for url
suffix = "&for=county:095&in=state:12&key=" + api_key

In [3]:
#Import csv
path = "../Resources/api_pulls.csv"
file = pd.read_csv(path)

#splits the column names by the "_" delimiter to sort year easier, makes array of first word in the split
frontsplit = file.columns.str.split('_').str[0]
years = []
#loop to take the year values out of split array
for element in frontsplit:
    if element != 'Formal':
        if element != 'Label':
            years.append(element)
#printing array to show years being looked at
print(years)


labels = []
for label in file["Label_Name"]:
    labels.append(label)
labels.append("State")
labels.append("County")
print(labels)

['2017', '2016', '2015', '2014', '2013']
['Households', 'Households (Family)', 'Number of Higher Ed Enrolled', 'Education of Some College - No Degree Attained', "Education of Associate's Degree Attained", "Education of Bachelor's Degree Attained", 'Education of Graduate or Professional Degree Attained', 'Households with Income Less than $10,000', 'Households with Income Between $10,000 and $14,999', 'Households with Income Between $15,000 and $24,999', 'Households with Income Between $25,000 and $34,999', 'Households with Income Between $35,000 and $49,999', 'Households with Income Between $50,000 and $74,999', 'Households with Income Between $75,000 and $99,999', 'Households with Income Between $100,000 and $149,999', 'Households with Income Between $150,000 and $199,999', 'Households with Income over $200,000', 'Median Household Income (in dollars)', 'Mean Household Income (in dollars)', 'Age of 20 to 24 years', 'Age of 25 to 34 years', 'Age of 35 to 44 years', 'Age of 45 to 54 years

In [4]:
#Begin For-Loops & Pulling APIs
response = {}
counter = 0
for year in years:
    url = baseurl + year + "/acs/acs1/profile?get="
    
    #building path to pull api with proper csv column
    filename = year + "_Variable"
    
    for row in file[filename]:
        url = url + row + ","
    
    #deletes trailing comma from the above loop
    url = url[:-1]
    url = url + suffix    
    
    response[counter] = requests.get(url).json()
    counter += 1

In [5]:
#initial dicitonary key is normal array index. This loop replaces the key with a year using "counter" to move through index
counter = 0
for year in years:
    response[year] = response.pop(counter)
    counter += 1
print(response)


{'2017': [['DP02_0001E', 'DP02_0002E', 'DP02_0057E', 'DP02_0062E', 'DP02_0063E', 'DP02_0064E', 'DP02_0065E', 'DP03_0052E', 'DP03_0053E', 'DP03_0054E', 'DP03_0055E', 'DP03_0056E', 'DP03_0057E', 'DP03_0058E', 'DP03_0059E', 'DP03_0060E', 'DP03_0061E', 'DP03_0062E', 'DP03_0063E', 'DP05_0009E', 'DP05_0010E', 'DP05_0011E', 'DP05_0012E', 'DP05_0013E', 'DP05_0014E', 'DP05_0015E', 'DP05_0016E', 'DP05_0017E', 'DP05_0037E', 'DP05_0038E', 'DP05_0039E', 'DP05_0044E', 'DP05_0052E', 'DP05_0057E', 'DP05_0059E', 'DP05_0060E', 'DP05_0061E', 'DP05_0062E', 'DP05_0070E', 'state', 'county'], ['460602', '303666', '117462', '169812', '114043', '206650', '101612', '32037', '21312', '46513', '44670', '67015', '86673', '51286', '55888', '25774', '29434', '54107', '79866', '105130', '227501', '189625', '179293', '84185', '65838', '95903', '43134', '18925', '831468', '286324', '2743', '69335', '1623', '107587', '15643', '4357', '10259', '633', '1348975', '12', '095']], '2016': [['DP02_0001E', 'DP02_0002E', 'DP02_0